In [1]:
#pip install pandas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [3]:
import sys
try:
    import pandas as pd
except ImportError as e:
    print("pandas is required for this example. Please install with conda or pip  and then try again.")
    sys.exit()

import sklearn
from sklearn import ensemble
import kmapper as km
from kmapper.plotlyviz import *
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")

In [4]:
import plotly.graph_objs as go
from ipywidgets import (HBox, VBox)

In [5]:
#pip install ripser

In [6]:
#pip install teaspoon

In [7]:
%matplotlib inline

In [8]:
#Colors
pl_brewer = [[0.0, '#006837'],
             [0.1, '#1a9850'],
             [0.2, '#66bd63'],
             [0.3, '#a6d96a'],
             [0.4, '#d9ef8b'],
             [0.5, '#ffffbf'],
             [0.6, '#fee08b'],
             [0.7, '#fdae61'],
             [0.8, '#f46d43'],
             [0.9, '#d73027'],
             [1.0, '#a50026']]

In [9]:
bgcolor = 'rgba(10,10,10, 0.9)'
y_gridcolor = 'rgb(150,150,150)'# on a black background the gridlines are set on  grey

In [10]:
df1 = pd.read_excel('19_1_1.xlsx', sheet_name='Hoja_1')
#df2 = pd.read_excel('20_1_0.xlsx', sheet_name='Hoja_1')
#df3 = pd.read_excel('21_1_1.xlsx', sheet_name='Hoja_1')

In [11]:
asignaturas2 = {'ALGEBRA LINEAL' : 1,
'BIOMATEMATICAS I' : 1,
'BIOMATEMATICAS II' : 1,
'CALCULO DIFERENCIAL' : 2,
'CALCULO DIFERENCIAL E INTEGRAL' : 2,
'CALCULO INTEGRAL' : 2,
'CALCULO VECTORIAL' : 2,
'DIFERENTIAL CALCULUS' : 2,
'ECUACIONES DIFERENCIALES' : 2,
'ESTADÍSTICA' : 3,
'ESTADÍSTICA DESCRIPTIVA' : 3,
'ESTADÍSTICA I' : 3,
'ESTADÍSTICA II' : 3,
'ESTADÍSTICA III' : 3,
'ESTADÍSTICA INFERENCIAL' : 3,
'INVESTIGACION DE OPERACIONES' : 4,
'LÓGICA Y RAZONAMIENTO MATEMÁTICO' : 1,
'MATEMATICAS AVANZADAS' : 4,
'MATEMATICAS BASICAS' : 1,
'MATEMATICAS ESPECIALES' : 4,
'MATEMÁTICAS I' : 1,
'MATEMÁTICAS II' : 1,
'MATEMÁTICAS III' : 2,
'METODOS MATEMATICOS' : 4,
'MÉTODOS NUMÉRICOS' : 4,
'MODELOS MATEMATICOS DE ADMON.' : 1,
'PROBABILIDAD Y ESTADISTICA' : 3,
'SISTEMAS LINEALES': 1}

In [12]:
df1['NOMBRE_ASIGNATURA'] = df1['NOMBRE_ASIGNATURA'].map(asignaturas2)

In [14]:
datos = df1
datos

,Unnamed: 0,NOTA,ESTAAPROCALI,NUMEFALLPARC,CODIGO_ESTUDIANTE,NOMBRE_ASIGNATURA,NIVEL_EN_PLAN,RANGO,FORMACION,CONTRATO,ESCALAFON
0,0,250,0,0,230,3,4,2,1,1,2
1,1,220,0,0,902,3,4,2,1,1,4
2,2,440,1,3,210,3,4,4,1,1,3
3,3,310,1,0,210,2,3,3,1,1,3
4,4,256,0,0,290,4,6,2,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
8212,8212,360,1,0,330,1,2,3,1,1,3
8213,8213,228,0,0,330,2,5,2,1,1,3
8214,8214,342,1,0,330,4,6,3,1,1,2
8215,8215,398,1,0,330,1,2,3,1,1,3


In [15]:
feature_names = [c for c in datos.columns if c not in ["Unnamed: 0",'ESTAAPROCALI','RANGO','NIVEL_EN_PLAN']]
X = np.array(datos[feature_names].fillna(0))
y = np.array(datos["ESTAAPROCALI"])
z = np.array(datos['FORMACION'])

In [16]:
datos.pivot_table(index = ['FORMACION'], aggfunc ='size')

FORMACION
0       2
1    7124
2    1091
dtype: int64

In [17]:
datos.pivot_table(index = ['ESTAAPROCALI'], aggfunc ='size')

ESTAAPROCALI
0    3068
1    5149
dtype: int64

In [18]:
# Create a 1-D lens with PCA-projection
mapper = km.KeplerMapper(verbose=0)
lens1 = mapper.fit_transform(X, projection="l2norm")
lens2 = mapper.fit_transform(X, projection=PCA(n_components=1))

scomplex = mapper.map(lens2,
                      X,
                      cover=km.Cover(n_cubes = 8,
                                     perc_overlap = 0.6),
                      clusterer=sklearn.cluster.KMeans(n_clusters=2,
                                                       random_state=3471))

In [19]:
#color_values according to z attribute
color_values = z - z.min()
my_colorscale = pl_brewer
kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex,
                                                                 color_values,
                                                                 color_function_name='Distance to x-min',
                                                                 colorscale=my_colorscale)

# assign to node['custom_tooltips']  the node label (0 for benign, 1 for malignant)
for node in kmgraph['nodes']:
    node['custom_tooltips'] = y[scomplex['nodes'][node['name']]]

In [20]:
plotly_graph_data = plotly_graph(kmgraph, graph_layout='fr', colorscale=my_colorscale,
                                 factor_size=2.5, edge_linewidth=0.5)
layout = plot_layout(title='Topological network representing <br>  Grades dataset for PCA',
                     width=600, height=550,
                     annotation_text=get_kmgraph_meta(mapper_summary),
                     bgcolor=bgcolor)

fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)
fw_hist = node_hist_fig(colorf_distribution, bgcolor=bgcolor,
                        y_gridcolor=y_gridcolor)
fw_summary = summary_fig(mapper_summary, height=300)
dashboard = hovering_widgets(kmgraph,
                             fw_graph,
                             ctooltips=True, # ctooltips = True, because we assigned a label to each
                                             #cluster member
                             bgcolor=bgcolor,
                             y_gridcolor=y_gridcolor,
                             member_textbox_width=600)

#Update the fw_graph colorbar, setting its title:

fw_graph.data[1].marker.colorbar.title = 'dist to<br>x-min'

In [21]:
VBox([fw_graph, HBox([fw_summary, fw_hist])])

    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(180,180,…

In [22]:
dashboard

    'data': [{'hoverinfo': 'none',
              'line': {'color'…

In [23]:
#Selecting the a cluster of a particular cube

In [24]:
dict = scomplex['nodes']
list2 = dict['cube4_cluster0']

In [25]:
data1=datos.iloc[list2,:]

In [26]:
data1

,Unnamed: 0,NOTA,ESTAAPROCALI,NUMEFALLPARC,CODIGO_ESTUDIANTE,NOMBRE_ASIGNATURA,NIVEL_EN_PLAN,RANGO,FORMACION,CONTRATO,ESCALAFON
15,15,300,1,0,460,2,3,3,1,1,3
21,21,260,0,0,550,1,2,2,1,1,2
22,22,310,1,0,470,1,2,3,1,1,3
30,30,250,0,0,600,4,6,2,1,1,3
54,54,160,0,0,600,4,6,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
8185,8185,239,0,0,600,1,1,2,1,1,3
8188,8188,190,0,2,580,1,1,1,1,1,3
8190,8190,230,0,0,540,1,1,2,1,1,2
8207,8207,110,0,2,540,1,1,1,1,1,2
